In [ ]:
import requests
from bs4 import BeautifulSoup
import re

import json
import time
import pandas as pd
from datetime import datetime

# 鉅亨網的API URL 最多可以抓到一個月該類型所有新聞的網址 每頁最多顯示30個網址
# https://news.cnyes.com/api/v3/news/category/tw_stock?startAt={start}&endAt={end}&limit=30

def get_newsid(start,end):
    newsid_list = []
    res = requests.get(f'https://news.cnyes.com/api/v3/news/category/tw_stock?startAt={start}&endAt={end}&limit=30')
    jsonData = json.loads(res.text)
    last_page = jsonData['items']['last_page']
    all_data =  jsonData['items']['data']
    
    for x in all_data:
        newsid_list.append(x['newsId'])
    
    for i in range(2,last_page+1):
        res = requests.get(f'https://news.cnyes.com/api/v3/news/category/tw_stock?startAt={start}&endAt={end}&limit=30&page={i}')
        jsonData = json.loads(res.text)
        all_data =  jsonData['items']['data']
    
        for x in all_data:
            newsid_list.append(x['newsId'])
        
    return newsid_list



def get_content(newsid_list):
    urls = []
    title = []
    author = []
    date_data = []
    content = []
    count = 0
    
    for newid in newsid_list:
        try:
            url = f"https://news.cnyes.com/news/id/{newid}?exp=a"
            res =  requests.get(url)
            soup = BeautifulSoup(res.text,'html.parser')
            urls.append(url)
            try:
                title.append(soup.find('h1').text)
            except:
                title.append('')
            try:
                author.append(soup.find('span',{'class':'_3lKe'}).text)
            except:
                author.append('')
            try:
                date_data.append(soup.find('time').text)
            except:
                date_data.append('')
            try:
                content.append(soup.find('div',{'class':'_1UuP'}).text)
            except:
                content.append('')
            count +=1
            print('處理了:',(count/len(newsid_list))*100,'%')
        except:
            print(url)
            continue
            
#        if count > 10:
#            break
    
    return urls, date_data, title, author, content
    



def storge(urls, date_data, title, author, content, start_y, start_m):
    pd_store= pd.DataFrame(columns = ['URL','Date','Title','Author','Content'])
    pd_store['URL'] = url
    pd_store['Date'] = date_data
    pd_store['Title'] = title
    pd_store['Author'] = author
    pd_store['Content'] = content
    pd_store.to_csv(f'gihun{start_y}{start_m}.csv', index=False)   


# 設定要抓取資料的月份 可寫for loop一次抓多個月
start_y, start_m, start_d = 2020, 7, 1 
end_y, end_m, end_d = 2020, 8, 1 
start = int(datetime(start_y, start_m, start_d).timestamp())
end = int(datetime(end_y, end_m, end_d).timestamp())


newsid_list = get_newsid(start,end)

url, date_data, title, author, content = get_content(newsid_list)

storge(url, date_data, title, author, content, start_y, start_m)  